In [34]:
import torch 
from torch import nn
from torch import optim
import torch.nn.functional as F
from blitz.modules import BayesianLinear

from copy import deepcopy

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.lin = nn.Sequential(
            nn.Linear(3, 6),
            BayesianLinear(6, 1))
        
    def forward(self, x):
        return(self.lin(x))

model = Model()
opt = optim.Adam(model.parameters())

model_clone = deepcopy(model) 
opt_clone = optim.Adam(model_clone.parameters())

inputs = torch.tensor([[1,1,1], [2,2,2], [3,3,3]]).float()
outputs = torch.tensor([[1], [2], [3]]).float()

predictions = model(inputs)
losses = F.mse_loss(predictions, outputs, reduction = "none")

opt.zero_grad()
losses.sum().backward()
opt.step()

In [35]:
for i in range(len(inputs)):
    opt_clone.zero_grad()
    losses[i].sum().backward() # How can I make losses backpropogate over model_clone instead of model?
    opt_clone.step()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.